---
## Lab Number : 8

### Title : *Logistic Regression with SparkMLib* 

### Background :  Logistic regression is a popular method to predict a categorical response. 

### Goal : predict if the client will subscribe (yes/no) a term deposit (variable y).

### Dataset

Full details @ : https://archive.ics.uci.edu/ml/datasets/bank+marketing

The dataset corresponds to :

Description of Variables :

---

## Just Some Preparation for plotting

In [33]:
# Set some parameters to apply to all plots. These can be overridden
# in each plot if desired
import matplotlib
# Plot size to 14" x 7"
matplotlib.rc('figure', figsize = (14, 7))
# Font size to 14
matplotlib.rc('font', size = 10)
# Do not display top and right frame lines
matplotlib.rc('axes.spines', top = False, right = False)
# Remove grid lines
matplotlib.rc('axes', grid = False)
# Set backgound color to white
matplotlib.rc('axes', facecolor = 'white')

## Some Useful Functions

In [34]:
def computeHistogram(df,column_name,nbins):
    """
    Histogram computation in Spark using flatmap and histogram for the selected column of a Spark DataFrame
    :param df : a Spark DataFrame object (not a python df)
    :param column_name : the column for which histogram is to be computed
    :param nbins : the nb. of bins for data accumulation
    """
    hist = df.select(column_name).rdd.flatMap(lambda x: x).histogram(nbins)
    return hist

def plotHistogram(hist,title,nticks=20):
    """
    Simple matplolib plotting of histogram
    :param hist : a histogram array containing (bins,counts)
    :param title : the histogram title
    """
    bins, counts = hist

    n      = len(counts) # nb of centers
    width  = 0.95 # width of bars
    centrs = np.arange(n)+0.5 # centers of bars

    # calculate the grid for the KDE
    grid=np.linspace(min(bins), max(bins), 1000)
    
    # Attempt to fit distribution using scipy kernel density estimator
    pdf=kde_scipy(counts, grid, bandwidth=0.5)
    
    fig, ax = plt.subplots()
    
    ax.bar(centrs, counts, width, color='#539caf', alpha = 0.75, align='center')
    ax.plot(grid, pdf , color='r')
    ax.set_ylabel('No. Of Objects')
    ax.set_title(title)
    
    # Compute the number ticks and labels to be show
    ticks  = np.arange(0, shape[0]+1, shape[0]/nticks)
    labels =[round(x, ndigits) for x in np.arange(min(bins), max(bins)+1, (max(bins)-min(bins))/nticks)]
    
    #ax.set_xticks(np.arange(n+1))
    #ax.set_xticklabels(bins)
    
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels)
        
    ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%i'))
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2e'))
    
    plt.show()

## Get a SparkSession object

In [27]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("LogisticRegrssion") \
    .getOrCreate()
    
print ('Spark version : ' , spark.version)

Spark version :  2.2.0


## Data Loading and Pre-Processing

In [28]:
# Load the Input Dataset into a Spark Dataframe
dataset_path='../../data/bank/'
%time df  = spark.read.option("header", "true").option("inferSchema", "true").csv(dataset_path+'banking.csv')
print('Number of rows in dataset : ' + str(df.count()))

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 711 ms
Number of rows in dataset : 41188


## Data Exploration

In [29]:
# What kind of attributes we have in the dataset?
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: double (nullable = true)
 |-- cons_price_idx: double (nullable = true)
 |-- cons_conf_idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr_employed: double (nullable = true)
 |-- y: integer (nullable = true)



In [30]:
# Simple statistics on age
df.select('age').describe().show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             41188|
|   mean| 40.02406040594348|
| stddev|10.421249980934071|
|    min|                17|
|    max|                98|
+-------+------------------+



In [31]:
# Simple statistics on Euribor
df.select('euribor3m').describe().show()

+-------+------------------+
|summary|         euribor3m|
+-------+------------------+
|  count|             41188|
|   mean|3.6212908128582826|
| stddev| 1.734447404851268|
|    min|             0.634|
|    max|             5.045|
+-------+------------------+



In [35]:
# Some histograms to see e.g : age distribution amongts sample 
age_hist=computeHistogram(df,'age',20)
plotHistogram(age_hist,'Age Distribution',nticks=20):

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 23.0 failed 1 times, most recent failure: Lost task 1.0 in stage 23.0 (TID 34, localhost, executor driver): java.io.IOException: Cannot run program "/home/asier/Software/anaconda3": error=13, Permission denied
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:163)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:89)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:65)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:128)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.computeOrRea